<a href="https://colab.research.google.com/github/JisungJang-AI-BIO/LAIDD_Developability/blob/main/1_Boltz_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 소개 및 설정

**실습: Boltz-2를 활용한 항체-항원 구조 예측**

**목표**: 이 노트북에서는 Boltz-2를 사용하여 Anti-GFP VHH (Nanobody)와 그 표적 항원인 GFP Antigen 결합 구조를 예측합니다. 아미노산 서열 정보만을 이용하여 복잡한 단백질 복합체의 3차원 구조를 생성하고 시각화하는 과정을 경험할 것입니다.

In [1]:
# 1단계: 기존 패키지를 모두 제거하여 깨끗한 상태를 보장합니다.
print("기존 PyTorch 및 Boltz 관련 패키지를 제거합니다...")
!pip uninstall torch torchvision torchaudio boltz -y -q

# 2단계: 모든 제약조건(boltz, torch==2.2.0 등)을 단일 명령어로 전달하여 한 번에 설치합니다.
# --extra-index-url을 사용하여 PyPI와 PyTorch 서버를 모두 검색하도록 합니다.
print("\nBoltz와 호환되는 모든 패키지를 동시에 설치합니다...")
!pip install boltz torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0 --extra-index-url https://download.pytorch.org/whl/cu121 -q

# 3단계: 설치된 버전을 최종 확인하여, torch가 의도한 대로 2.2.0으로 설치되었는지 검증합니다.
print("\n설치된 패키지 버전을 확인합니다.")
!pip show torch torchvision

기존 PyTorch 및 Boltz 관련 패키지를 제거합니다...

Boltz와 호환되는 모든 패키지를 동시에 설치합니다...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 32.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuequivariance-ops-cu12 0.5.1 requires nvidia-cublas-cu12>=12.5.0, but you have nvidia-cublas-cu12 12.1.3.1 which is incompatible.
distributed-ucxx-cu12 0.42.0 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.0 which is incompatible.
dask-cuda 25.2.0 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.0 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have n

In [ ]:
import yaml
import datetime
import os
import glob
import py3Dmol

# 1. 입력 데이터 변경: Anti-GFP VHH와 GFP 단백질 서열을 정의합니다.
# ---------------------------------------------------------------------------------
# Anti-GFP VHH (Nanobody, 약 116 아미노산)
vhh_sequence = "QVQLQESGGGLVQAGGSLRLSCAASGRIFSSDMGWFRQAPGKEREFVAAISWSGGRTYYADSVKGRFTISQDNAKNTVYLQMNSLKPEDTALYYCAADLSTYWGQGTQVTVSS"

# GFP Antigen (약 238 아미노산)
antigen_sequence = "MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITLGMDELYK"

# 새로운 출력 파일 이름
output_name = "VHH_GFP_prediction"


# 2. YAML 파일 생성: VHH는 Heavy chain으로, GFP는 Antigen으로 지정합니다.
# ---------------------------------------------------------------------------------
input_data = {
    'sequences': [
        {
            'protein': {
                'id': 'H',  # VHH (Nanobody)
                'sequence': vhh_sequence
            }
        },
        {
            'protein': {
                'id': 'A',  # GFP (Antigen)
                'sequence': antigen_sequence
            }
        }
    ],
    'name': output_name
}

yaml_path = f"{output_name}.yaml"
with open(yaml_path, 'w') as f:
    yaml.dump(input_data, f, indent=2, sort_keys=False)

print(f"✅ 새로운 YAML 파일 생성 완료: {yaml_path}")


# 3. Boltz 예측 실행: 충돌 방지를 위해 매번 새로운 출력 폴더를 지정합니다.
# ---------------------------------------------------------------------------------
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
output_dir = f"boltz_run_{timestamp}"

print(f"✅ 결과가 저장될 폴더: {output_dir}")
print("\nBoltz-2 예측을 시작합니다...")

!boltz predict {yaml_path} \
    --use_msa_server \
    --output_format pdb \
    --out_dir {output_dir}

print("\n--- 예측 프로세스 완료 ---")


# 4. 결과 시각화
# ---------------------------------------------------------------------------------
print("\n결과 시각화를 시작합니다...")

pdb_files = glob.glob(os.path.join(output_dir, '**', '*.pdb'), recursive=True)

try:
    if not pdb_files:
        raise FileNotFoundError(f"'{output_dir}' 폴더에서 .pdb 파일을 찾을 수 없습니다.")

    pdb_files.sort()
    pdb_file_path = pdb_files[0]
    print(f"✅ 발견된 PDB 파일: {pdb_file_path}")

    with open(pdb_file_path, 'r') as f:
        pdb_data = f.read()

    view = py3Dmol.view(width=800, height=600)
    view.addModel(pdb_data, "pdb")
    # VHH는 'H', Antigen은 'A' 체인으로 시각화
    view.setStyle({'chain':'H'}, {'cartoon': {'color':'magenta'}})
    view.setStyle({'chain':'A'}, {'cartoon': {'color':'green'}})
    view.zoomTo()
    view.show()

except FileNotFoundError as e:
    print(f"오류: {e}")
    print("예측이 끝까지 완료되지 않았을 수 있습니다. 위 로그에서 진행률(%)이 올라갔는지 확인해주세요.")
except Exception as e:
    print(f"시각화 중 오류가 발생했습니다: {e}")

✅ 새로운 YAML 파일 생성 완료: VHH_GFP_prediction.yaml
✅ 결과가 저장될 폴더: boltz_run_20250705_061542

Boltz-2 예측을 시작합니다...
Checking input data.
Running predictions for 1 structure
Processing input data.
  0% 0/1 [00:00<?, ?it/s]Generating MSA for VHH_GFP_prediction.yaml with 2 protein entities.

  0% 0/300 [00:00<?, ?it/s]
SUBMIT:   0% 0/300 [00:00<?, ?it/s]
PENDING:   0% 0/300 [00:00<?, ?it/s]Sleeping for 10s. Reason: PENDING

RUNNING:   0% 0/300 [00:10<?, ?it/s]
RUNNING:   3% 10/300 [00:10<05:05,  1.05s/it]Sleeping for 8s. Reason: RUNNING

RUNNING:   3% 10/300 [00:18<05:05,  1.05s/it]
RUNNING:   6% 18/300 [00:18<04:52,  1.04s/it]Sleeping for 5s. Reason: RUNNING

RUNNING:   6% 18/300 [00:23<04:52,  1.04s/it]
RUNNING:   8% 23/300 [00:23<04:47,  1.04s/it]Sleeping for 9s. Reason: RUNNING

COMPLETE:   8% 23/300 [00:33<04:47,  1.04s/it]
COMPLETE: 100% 300/300 [00:33<00:00,  8.99it/s]

  0% 0/300 [00:00<?, ?it/s]
SUBMIT:   0% 0/300 [00:00<?, ?it/s]
PENDING:   0% 0/300 [00:00<?, ?it/s]Sleeping for 8s. Reaso